In [ ]:
# 필요한 라이브러리 import

import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [ ]:
# 데이터셋 다운로드 및 전처리

from datasets import load_dataset

dataset = 'lifestyle'
datasplit = 'dev'

collection_dataset = load_dataset("colbertv2/lotte_passages", dataset)
collection = [x['text'] for x in collection_dataset[datasplit + '_collection']]

queries_dataset = load_dataset("colbertv2/lotte", dataset)
queries = [x['query'] for x in queries_dataset['search_' + datasplit]]

f'Loaded {len(queries)} queries and {len(collection):,} passages'

In [ ]:
# 쿼리 한 개와 구절 한 개를 검사하여 제대로 수행되었는지 확인해 보겠습니다.
print(queries[0])
print(collection[0])

## 1. Indexing

빠른 실습을 위해 처음 2000개의 구절만 색인

In [4]:
# 기본적인 상수 정의

nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
max_id = 2000

index_name = f'{dataset}.{datasplit}.{nbits}bits'

In [ ]:
# ID가 2000 미만인 구절이 포함되지 않은 쿼리를 필터링

answer_pids = [x['answers']['answer_pids'] for x in queries_dataset['search_' + datasplit]]
filtered_queries = [q for q, apids in zip(queries, answer_pids) if any(x < max_id for x in apids)]

f'Filtered down to {len(filtered_queries)} queries'


GPU를 하나만 사용한다고 가정하면, 색인 완료까지 약 3분이 소요

In [ ]:
#  모델은 ColBERT를 설치하면 바로 사용할 수 있는 colbertv2.0을 그대로 사용

checkpoint = 'colbert-ir/colbertv2.0'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection[:max_id], overwrite=True)

In [ ]:
indexer.get_index() # You can get the absolute path of the index, if needed.

## 2. Search


In [ ]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

In [ ]:
query = filtered_queries[0] # try with an in-range query or supply your own
print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

In [ ]:
query = filtered_queries[1] # try with an in-range query or supply your own
print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

한국어 데이터에 대한 색인/검색

In [ ]:
# 30개의 과학 상식 문서 샘플
sample_ko_collection = [
  "금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.",
  "공이 땅에서 굴러가고 있습니다. 이때, 공이 움직이는 방향과 같은 방향으로 힘이 공을 밀어줍니다. 이렇게 힘이 가해지면 공은 원래 움직이던 방향으로 더 빠르게 움직입니다. 이는 힘이 공에 가해져서 공의 운동에 영향을 주기 때문입니다. 힘은 질량에 가해지는 힘과 방향에 따라 운동에 영향을 줄 수 있습니다. 따라서, 공이 힘을 받으면 힘이 가해진 방향과 같은 방향으로 더 빠르게 움직이게 됩니다.",
  "전류가 통하는 와이어 주변의 자기장 선은 원 형태를 띠게 됩니다. 이는 전류가 흐르는 와이어 주변에 자기장이 형성되기 때문입니다. 자기장은 전류의 방향에 따라 원형으로 형성되며, 와이어 주변에는 자기장 선이 형성됩니다. 이 자기장 선은 와이어 주변에 일정한 거리 간격으로 분포되어 있으며, 전류의 세기에 따라 자기장의 세기도 달라집니다. 따라서 전류가 통하는 와이어 주변의 자기장 선은 원 형태를 띠게 됩니다.",
  "일반적으로 특정 동물들을 이주하게 만드는 두 가지 환경 변화는 계절 변화와 먹이 감소입니다. 계절 변화는 동물들이 생존에 필요한 조건이 변화하는 것을 의미합니다. 예를 들어, 겨울철에는 온도가 급격히 낮아지고 먹이가 부족해지는 등의 환경 변화가 발생합니다. 이에 따라 일부 동물들은 더 따뜻한 지역으로 이주하게 됩니다. 먹이 감소는 동물들이 생존에 필요한 먹이 자원이 줄어드는 것을 의미합니다. 이는 자연적인 이유로 발생할 수도 있고, 인간의 활동에 의해 발생할 수도 있습니다. 예를 들어, 산불이 발생하여 식물이 모두 타버리거나, 개발로 인해 서식지가 파괴되는 등의 상황에서 동물들은 먹이를 찾기 위해 다른 지역으로 이주하게 됩니다. 이러한 환경 변화는 동물들의 생존에 큰 영향을 미치며, 이주는 동물들이 적절한 생존 환경을 찾기 위한 전략 중 하나입니다.",
  "인간의 몸은 다양한 부위에서 수정이 발생합니다. 그 중에서도 일반적으로 수정이 일어나는 곳은 난관입니다. 난관은 인간의 생식기계에서 중요한 역할을 담당하며, 정자와 난자의 만남이 이루어지는 곳입니다. 난관은 여러 개의 작은 관으로 이루어져 있으며, 그 안에서 정자와 난자가 만나 결합되어 임신이 시작됩니다. 따라서 난관은 인간의 생식과 번식에 있어서 매우 중요한 역할을 수행합니다.",
  "학교 버스는 동일한 거리에 놓여 있을 때, 가장 큰 인력을 가질 것이다. 이는 학교 버스 간에 존재할 것이다. 학교 버스는 많은 학생들을 운송하고, 학교로 가는 길에 많은 사람들을 만난다. 따라서, 학교 버스는 많은 사람들의 관심과 주목을 받을 것이다. 또한, 학교 버스는 학교의 상징이기도 하며, 학생들에게 안전하고 편리한 이동 수단을 제공한다. 따라서, 학교 버스는 동일한 거리에 놓여 있을 때, 가장 큰 인력을 가지고 있을 것이다.",
  "머클-담고르 해시 함수인 Hh를 가정하고, 이를 MAC으로 구성하는 방법은 S(k,m) = H(k || m)입니다. 이 MAC의 보안이 취약한 이유는 H(k || m)가 주어지면 누구나 모든 w에 대해 H(k || m || PB || w)를 계산할 수 있다는 점입니다. 이는 MAC의 인증 기능을 악용하여 원하지 않는 데이터를 인증하는 공격을 할 수 있게 됩니다. 따라서 이 MAC은 보안 측면에서 취약하다고 할 수 있습니다.",
  "디엔에이 리가아제는 정상적인 디엔에이 복제에서 조각을 결합하는 효소입니다. 이 효소는 DNA 분자의 양 끝에 있는 조각을 인식하고 결합하여 새로운 DNA 분자를 생성합니다. 디엔에이 리가아제는 DNA 복제 과정에서 중요한 역할을 담당하며, DNA의 복제 정확성과 안정성을 유지하는 데에 기여합니다. 이러한 효소는 생물체의 성장과 발달, 유전 정보의 전달에 핵심적인 역할을 수행합니다. 디엔에이 리가아제는 생물학적인 과정에서 필수적인 효소로 알려져 있으며, 다양한 생물체에서 발견됩니다. 이러한 효소의 기능과 작용 메커니즘은 생물학 연구의 중요한 주제 중 하나입니다. 디엔에이 리가아제는 DNA 분자의 구조와 기능을 이해하는 데에 중요한 역할을 하며, 유전자 조작과 유전자 치료 등의 응용 분야에서도 활용될 수 있습니다. 디엔에이 리가아제에 대한 연구는 현재도 진행 중이며, 그 결과는 생명과학 분야에 큰 영향을 미칠 것으로 기대됩니다.",
  "양적형질유전자좌 분석은 유전 교차에서 복잡형질과 관련된 염색체 영역을 식별하는데 사용됩니다. 이 분석은 유전자의 위치와 기능을 이해하는 데 도움이 됩니다. 양적형질유전자좌 분석은 유전적 다양성을 연구하고, 특정 형질이나 질병과 연관된 유전자를 찾는 데 사용됩니다. 이를 통해 우리는 유전적인 원인과 결과를 이해하고, 질병 예방이나 개선을 위한 전략을 개발할 수 있습니다. 양적형질유전자좌 분석은 생명과학 분야에서 중요한 도구로 사용되며, 유전학 연구의 발전에 큰 기여를 하고 있습니다.",
  "램프의 개수가 회로에 병렬로 연결될 때, 전원의 전류는 감소한다. 이는 병렬 연결된 램프들이 전원의 전압을 공유하기 때문이다. 각 램프는 동일한 전압을 받으며, 따라서 전류는 분할된다. 따라서 램프의 개수가 늘어남에 따라 전원의 전류는 감소한다. 이는 전류의 분배 법칙에 따른 결과이다. 램프의 개수가 늘어남에 따라 전류의 분배가 일어나므로, 각 램프에 흐르는 전류는 줄어들게 된다. 따라서 전원의 전류는 감소한다.",
  "오빠와 여동생이 시소에서 균형을 잡을 수 있는 이유는 토크입니다. 토크는 물체에 작용하는 힘의 회전력을 의미합니다. 시소에서 오빠와 여동생은 서로 반대 방향으로 힘을 가하면서 균형을 유지합니다. 오빠가 시소를 오른쪽으로 밀면, 여동생은 왼쪽으로 밀어지는 힘을 받게 됩니다. 이렇게 반대 방향으로 가해지는 힘들이 서로 상쇄되어 균형을 유지할 수 있습니다. 토크는 두 힘의 크기와 힘의 지점 사이의 거리에 비례합니다. 따라서 오빠와 여동생이 시소에서 균형을 잡을 수 있는 이유는 토크의 원리에 의해 가능합니다.",
  "지구와 달이 상호 작용하는 방법에는 다양한 예시가 있습니다. 첫째로, 만유인력의 법칙에 따라 지구와 달은 서로에게 인력을 행사합니다. 이로 인해 달은 지구 주변을 돌면서 우리에게 달의 다양한 달기를 보여줍니다. 둘째로, 달은 지구의 조력을 받아 조금씩 멀어지고 있습니다. 이는 달의 궤도가 조금씩 넓어지는 현상으로 알려져 있습니다. 셋째로, 달은 지구의 조력을 받아 조금씩 지구와의 회전 주기가 일치하게 됩니다. 이로 인해 우리는 항상 같은 면만을 보게 되는 달의 양면성을 경험할 수 있습니다. 마지막으로, 달은 지구의 조력을 받아 조금씩 지구의 자전 주기를 늦추고 있습니다. 이는 지구의 자전 속도가 달의 조력에 의해 점차 감소하는 현상으로 알려져 있습니다. 이렇게 다양한 방법으로 지구와 달은 상호 작용하고 있습니다. 그러나 지구의 계절은 지구의 자전축의 기울기와 태양의 각도에 의해 결정되는 현상으로, 지구와 달의 상호 작용과는 직접적인 연관이 없습니다.",
  "지구 온난화는 전 세계적인 기온 상승을 의미합니다. 이는 지구 대기와 해양의 온도가 상승하여 지구의 기후가 변화하는 현상입니다. 지구 온난화는 인간의 활동으로 인해 발생하는 온실가스의 배출이 증가함에 따라 가속화되고 있습니다. 이러한 온난화는 지구의 생태계와 생물 다양성에 부정적인 영향을 미치고 있으며, 극지방의 빙하와 빙산의 녹음으로 인해 해수면 상승과 같은 심각한 문제를 야기할 수 있습니다. 따라서 지구 온난화에 대한 대응과 지속 가능한 에너지 및 자원 이용이 중요한 과제입니다.",
  "과일샐러드는 성분을 분리할 수 있는 혼합물입니다. 이 혼합물은 다양한 과일들로 구성되어 있습니다. 과일샐러드에는 사과, 바나나, 딸기, 포도 등 다양한 과일이 들어갈 수 있습니다. 이러한 과일들은 각각 독립적으로 존재하며, 혼합되어 있을 때>도 각각의 특징과 맛을 유지합니다. 따라서, 과일샐러드는 성분을 분리할 수 있는 혼합물 중 하나입니다. 이러한 혼합물은 맛과 영양을 동시에 즐길 수 있는 좋은 선택입니다.",
  "파리는 한 동물로서 여섯 개의 다리를 가지고 있습니다. 파리는 곤충 중 하나로서 작은 크기와 날개를 가지고 있습니다. 파리는 주로 썩은 음식이나 동물 배설물에 서식하며, 사람들에게는 귀찮은 해충으로 알려져 있습니다. 파리는 빠른 속도로 날아다니며, 사람들이나 동물들에게 짜증을 주기도 합니다. 그러나 파리는 생태계에서 중요한 역할을 하고 있으며, 다른 동물들의 먹이로서도 기여합니다. 따라서 파리는 여섯 개의 다리를 가진 동물 중 하나로서 가장 가능성이 높은 후보입니다.",
  "솔레노이드는 전기 전류가 흐르는 와이어로 구성된 장치입니다. 이 문제에서는 10 암페어의 전류가 흐르고 와이어가 1000번 감겨있는 길이 30cm 지름 3cm의 솔레노이드의 자기장에 저장된 에너지를 계산해야 합니다. 솔레노이드의 자기장에 저장된 에너지는 전류의 크기, 와이어의 감긴 횟수, 길이, 지름 등의 요소에 의해 결정됩니다. 이 문제에서는 솔레노이드의 자기장에 저장된 에너지가 0.15 줄이라고 주어졌습니다. 따라서, 이 문제에서 요구하는 답은 0.15 줄입니다.",
  "미국의 많은 도시들이 친환경 공동체를 조성하고 있습니다. 이러한 친환경 공동체는 주택, 아파트 및 인근의 사업체들로 구성되어 있으며, 에너지 사용 절감을 목표로 합니다. 공동체 내에는 식료품점, 레스토랑, 영화관 등이 위치해 있어 주민들이 도보로 이동할 수 있습니다. 이로 인해 사람들의 등교 및 통근 거리가 짧아지고, 환경에 대한 부담도 줄어듭니다. 또한, 친환경 공동체의 많은 건물들은 재생 및 재활용 가능한 재료를 사용하여 지어졌습니다. 태양 에너지를 이용하여 건물 내부를 쾌적한 온도로 유지하고, 나무와 식물과 같은 자연물은 신중하게 배치되어 그늘을 제공하고 온도를 조절합니다. 또한, 공동체 내에는 주민들이 음식과 꽃을 재배할 수 있는 정원도 마련되어 있습니다. 하지만, 친환경 공동체의 사람들이 재생 불가 자원>을 사용하는 경우도 있습니다. 예를 들어, 가솔린으로 움직이는 자동차를 운전할 때는 환경에 부정적인 영향을 미칠 수 있습니다. 따라서 친환경 공동체의 주민들은 자동차 대신 대중교통이나 자전거를 이용하여 환경을 보호하는 노력을 해야 합니다.",
  "표 안의 정보를 통해 학생들은 식물 같은 생물체들이 더 많다는 결론을 내릴 수 있습니다. 표에는 다양한 생물체들의 수가 나열되어 있으며, 이를 비교해보면 식물의 수가 다른 생물체들보다 더 많은 것을 알 수 있습니다. 이는 식물이 다른 생물체들에 비해 번식력이 높거나, 생태계에서 더 중요한 역할을 한다는 것을 시사합니다. 따라서, 표 안의 정보를 통해 학생들은 식물 같은 생물체들이 더 많다는 결론을 내릴 수 있습니다.",
  "하구퇴적지는 많은 종이 알을 낳고 새끼를 키우는 곳으로 알려져 있습니다. 이러한 곳은 바다의 보육원으로 불리며, 많은 생물들에게 안전한 은신처를 제공합니다. 하구퇴적지는 풍부한 은신처를 가지고 있어 다양한 해양 생물들이 서식할 수 있습니다. 또한, 하구퇴적지는 농축된 영양분을 가지고 있어 생물들이 영양을 공급받을 수 있습니다. 이러한 특징들이 하구퇴적지가 바다의 보육원 역할을 가장 잘 뒷받침하는 이유입니다.",
  "행성의 암석권은 지각과 맨틀 상부의 단단한 암석 물질로 구성되어 있습니다. 이 암석들은 지구의 표면을 형성하는데 중요한 역할을 합니다. 지각은 행성의 외부 층으로, 흙, 모래, 암석 등으로 이루어져 있습니다. 맨틀은 지각 아래에 위치하며, 암석으로 이루어진 두꺼운 층입니다. 이러한 암석들은 행성의 구조와 지질학적 현상에 영향을 미치는 중요한 역할을 합니다. 암석권은 행성의 내부 구조를 이해하는데 도움을 주는 중요한 정보를 제공합니다.",
  "칼리시바이러스과는 양성 가닥 RNA 바이러스로 복제됩니다. 이 바이러스는 호스트 세포에 침투하여 그들의 기계적인 시스템을 이용하여 복제됩니다. 칼리시바이러스과의 바이러스는 RNA 복제 효소를 이용하여 자신의 RNA를 복사하고 새로운 바이러스 입자를 형성합니다. 이러한 복제과정은 호스트 세포의 생리학적 기능을 간섭하고, 세포의 죽음을 유발할 수 있습니다. 칼리시바이러스과의 바이러스는 또한 호스트 세포의 면역 시스템을 회피하기 위해 다양한 전략을 사용합니다. 이러한 복제과정은 바이러스의 생존과 번식을 위해 중요한 역할을 합니다.",
  "전자와 양성자는 300 K에서 1.0 T 자기장에 위치해 있습니다. 이러한 조건에서 전자와 양성자의 평형 분극 비율은 820입니다. 평형 분극 비율은 전자의 분극 에너지(pe)를 양성자의 분극 에너지(pH)로 나눈 값입니다. 이 값은 820으로 계산됩니다. 평형 분극 비율은 자기장의 강도와 온도에 따라 변할 수 있으며, 이 경우에는 1.0 T 자기장과 300 K 온도에서 계산된 값입니다. 이러한 결과는 전자와 양성자의 상호작용과 에너지 분포에 대한 중요한 정보를 제공합니다.",
  "중성자는 원자핵의 구성 요소 중 하나로, 어떤 원자핵에 질량을 더하는 입자입니다. 중성자는 전하를 가지지 않고, 양성자와 함께 원자핵의 중심에 위치하며, 원자핵의 안정성과 핵반응에 중요한 역할을 합니다. 중성자는 핵융합 반응에서 에너지를 생성하는 핵반응의 주요한 원인 중 하나이기도 합니다. 중성자는 핵무기의 동력원이 되기도 하며, 핵발전소에서도 중성자를 이용하여 전기를 생산합니다. 중성자는 핵물리학과 핵공학 등 다양한 분야에서 중요한 개념이며, 핵에 질량을 더하는 입자로서 그 역할과 특성을 가장 잘 설명합니다.",
  "후두신경의 손상이 의심될 때 성대주름에 후두경검사를 실시하면, 대체로 손상된 쪽의 성대주름이 긴장되어 있으며 내전과 외전의 중간상태에 놓여 있는 것이 확인된다. 이러한 현상은 윤상갑상근이 아직 기능하고 있지만 피열간근이 약해진 상태이기 때문이다. 윤상갑상근은 성대주름을 긴장시키는 역할을 하며, 피열간근은 성대주름을 외전하거나 내전하는 역할을 한다. 따라서, 후두신경의 손상으로 인해 윤상갑상근은 여전히 작동하지만 피열간근이 약해져서 성대주름이 내전과 외전의 중간상태에 놓이게 된다. 이러한 결과가 후두경검사를 통해 확인될 수 있다.",
  "식물의 씨앗은 생명력을 지니고 있으며, 다양한 기능을 수행합니다. 그 중 한 가지 기능은 초기 발달에 필요한 음식을 제공하는 것입니다. 씨앗은 작고 작은 식물의 모습을 담고 있으며, 이 모습은 식물이 자라기 위해 필요한 영양분을 포함하고 있습니다. 씨앗은 흙 속에 심어지면 물과 영양분을 흡수하여 발아하고 자라나게 됩니다. 이렇게 씨앗은 식물의 초기 발달에 필요한 음식을 제공하여 식물이 건강하게 자라날 수 있도록 도와줍니다. 씨앗은 또한 환경 조건에 따라 발아하는 시기를 조절할 수도 있으며, 씨앗의 외부 껍질은 식물을 보호하고 안전하게 자라날 수 있도록 도와줍니다. 따라서 식물의 씨앗은 생명력을 유지하고 번식을 위한 중요한 역할을 수행하는 것입니다.",
  "강은 계곡과 협곡을 형성하는 주요한 요소입니다. 강은 수많은 물들이 모여 흐르는 자연적인 흐름입니다. 강은 지형을 깊게 파고들어 계곡과 협곡을 형성합니다. 강물은 강력한 힘을 가지고 있어서 흙과 바위를 침식시키고, 이로 인해 계곡과 협곡이 형성됩니다. 강은 자연의 힘과 영향력을 보여주는 중요한 지형 요소입니다.",
  "기억 상실은 대뇌의 기능 장애로 인해 발생할 가능성이 가장 높습니다. 대뇌는 인간의 중추신경계에서 가장 중요한 역할을 담당하며, 인지, 기억, 감정 등 다양한 기능을 조절합니다. 따라서 대뇌의 어느 부분에서 문제가 발생하면 기억 상실이나 기타 인지 기능의 장애가 발생할 수 있습니다. 대뇌는 뇌의 가장 큰 부분으로, 뇌의 표면에 위치한 뇌피질과 그 아래에 있는 심층뇌로 구성되어 있습니다. 이러한 대뇌의 구조와 기능이 원활하게 작동하지 않으면 기억 상실이 발생할 수 있습니다. 따라서 대뇌는 기억 상실과 관련된 가장 중요한 부분으로 간주됩니다.",
  "폭풍을 쫓는 자들은 폭풍을 관찰하고 측량하기 위해 폭풍에 가까이 다가가는 사람들입니다. 이들은 폭풍의 움직임, 구조, 성질 등을 연구하여 폭풍 형성 이론을 발전시키는 데 기여하고 있습니다. 그 중에서도 폭풍 시작 시 수집한 데이터가 폭풍 형성 이론을 가장 많이 수정하는 데 도움이 되었습니다. 이 데이터는 폭풍의 초기 조건과 환경 요소를 포착하여 폭풍의 발달과 진화에 대한 이해를 높이는 데 도움이 되었습니다. 폭풍을 쫓는 자들은 이러한 데이터를 분석하고 해석하여 폭풍 형성 이론을 수정하고 개선하는 데 중요한 역할을 하고 있습니다. 이를 통해 우리는 폭풍의 원리와 특성을 더욱 깊이 이해할 수 있게 되었습니다.",
  "자동차에서 사용한 엔진 오일을 폐기하는 것과 관련이 없는 환경적 리스크는 물에 녹아서 산성비를 일으키는 것입니다. 하지만 이는 엔진 오일을 폐기하는 것과 직접적인 연관이 없는 문제입니다. 엔진 오일은 자동차의 엔진을 윤활하고 성능을 향상시키는 역할을 합니다. 따라서 엔진 오일을 폐기하는 것은 환경적인 문제를 야기할 수 있습니다. 엔진 오일은 잘못 처리되면 지하수나 물에 녹아들어 산성비를 일으킬 수 있습니다. 이는 식물과 동물의 생태계에 영향을 미칠 수 있으며, 물질의 독성으로 인해 생물 다양성을 감소시킬 수도 있습니다. 따라서 엔진 오일을 폐기할 때는 환경에 안전하게 처리하는 것이 중요합니다. 정부와 기업은 엔진 오일을 안전하게 처리하기 위한 규제와 시설을 마련하여 환경적인 리스크를 최소화해야 합니다.",
  "특정한 상염색체 열성 특성은 인구의 1%에서 표현됩니다. 이는 하디-바인베르크 평형 상태에서의 인구의 특성입니다. 하디-바인베르크 평형 상태란, 유전자 주파수가 변하지 않는 상태를 의미합니다. 따라서, 특정한 상염색체 열성 특성을 표현하지 않는 매개체의 비율은 인구의 18%입니다. 이는 하디-바인베르크 평형 상태에서의 인구의 특성을 고려하여 계산된 값입니다."
]

In [ ]:
checkpoint = 'colbert-ir/colbertv2.0'
index_name = 'sample_ko_org'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=sample_ko_collection, overwrite=True)

In [ ]:
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=sample_ko_collection)

# Find the top-3 passages for this query
query = '지구 온난화에 대해 알려줘'
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

## 3. Training

LLM을 활용한 생성데이터로 ColBERT를 학습한 후 더 나은 성능의 모델을 활용하는 방법 실습

### 3.1 Pseudo train dataset 생성

In [ ]:
from openai import OpenAI
import os
import json
os.environ["OPENAI_API_KEY"] = ""
llm_model = "gpt-3.5-turbo-1106"
client = OpenAI()

In [ ]:
augment_instruct = """
## Role
가상 데이터 생성기

## Instructions
- 주어진 레퍼런스 정보를 보고 이 정보가 도움이 될만한 질문을 가상으로 3개 생성해줘.
- 아래 JSON 포맷으로 생성해줘.

## Output format
{"questions": [$question1, $question2, $question3]}
"""

In [ ]:
def autogen(messages, llm_model):
  result = client.chat.completions.create(
            model=llm_model,
            messages=messages,
            temperature=0.5,
            response_format={"type": "json_object"},
            timeout=10,
            seed=1
    )
  return result

In [ ]:
import random

triples_data = []
query_data = []
q_idx = 0
max_c_idx = len(sample_ko_collection) - 1
for c_idx, sample in enumerate(sample_ko_collection):
  messages = [
    {"role": "system", "content": augment_instruct},
    {"role": "user", "content": sample}
  ]
  result = autogen(messages, llm_model)
  print(result.choices[0].message.content)
  try:
    content = json.loads(result.choices[0].message.content)
    for q in content["questions"]:
      query_data.append(q)
      mc_idx = random.randint(0, max_c_idx)
      while mc_idx == c_idx:
        mc_idx = random.randint(0, max_c_idx)
      triples_data.append(f'{q_idx}, {c_idx}, {mc_idx}')
      q_idx += 1
  except Exception as e:
    continue


print(triples_data)
print(query_data)


In [ ]:
# ColBERT 학습을 위하여 학습 데이터를 파일에 저장
collection_file = './collection.tsv'
query_file = './query.tsv'
triples_file = './triples'

with open(collection_file, 'w') as f:
  for i,item in enumerate(sample_ko_collection):
    f.write(f'{i}\t{item}\n')

with open(query_file, 'w') as f:
  for i,item in enumerate(query_data):
    f.write(f'{i}\t{item}\n')

with open(triples_file, 'w') as f:
  for i,item in enumerate(triples_data):
    f.write(f'[{item}]\n')

In [ ]:
!cat triples

In [ ]:
!ls -l

### 3.2 새로 만든 데이터로 모델 학습
90개의 소규모 데이터로 학습 진행

In [ ]:
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert import Trainer

if __name__=='__main__':
    with Run().context(RunConfig(nranks=1, experiment="sample_ko_new")):

        config = ColBERTConfig(
            bsize=24,
            root="./experiments",
        )

        trainer = Trainer(
            triples=triples_file,
            queries=query_file,
            collection=collection_file,
            config=config,
        )

        # Pretrained model을 한국어 기반 모델로 설정해 준다.
        checkpoint_path = trainer.train(checkpoint='hunkim/sentence-transformer-klue')
        #checkpoint_path = trainer.train()

        print(f"Saved checkpoint to {checkpoint_path}...")

In [ ]:
# 학습된 모델의 위치
!find experiments/sample_ko_new -name colbert

학습된 모델로 다시 색인 및 검색

In [ ]:
# 위에서 확인한 학습된 모델의 위치를 checkpoint에 넣어줌
checkpoint = 'experiments/sample_ko_new/none/2024-01/24/07.33.10/checkpoints/colbert'
index_name = 'sample_ko_new'
nbits = 2

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=sample_ko_collection, overwrite=True)

In [ ]:
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=sample_ko_collection)

# Find the top-3 passages for this query
query = '지구 온난화에 대해 알려줘'
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

#Reference

## Required Package

colbert-ir==0.2.14 <br>
torch==1.13.1 <br>
faiss-gpu==1.7.0 <br>
openai==1.7.2
